In [33]:
import csv

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

In [34]:
train_df = pd.read_csv('/hdd/datasets/sarcasm/train-balanced-sarcasm.csv')

In [38]:
train_df = train_df.drop(train_df.columns[[2,3,4,5,6,7,8,9]], axis=1)
print(train_df.head())

comments = train_df['comment'].tolist()
comments = [i.split(' ') for i in comments if isinstance(i, str)]

   label                                            comment
0      0                                         NC and NH.
1      0  You do know west teams play against west teams...
2      0  They were underdogs earlier today, but since G...
3      0  This meme isn't funny none of the "new york ni...
4      0                    I could use one of those tools.


In [39]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(comments)

In [42]:
print(comments[:2])
print(tokenizer.texts_to_sequences(comments[:2]))

[['NC', 'and', 'NH.'], ['You', 'do', 'know', 'west', 'teams', 'play', 'against', 'west', 'teams', 'more', 'than', 'east', 'teams', 'right?']]
[[6425, 5], [6, 48, 65, 1182, 749, 163, 235, 1182, 749, 53, 75, 1363, 749, 194]]
